In [1]:
from parsing.config import Config
from parsing.directory import list_files_in_directory
from parsing.parse_xls import XLSParser, RawWood
from parsing.volumes_table import Volume
from wood_objects.wood import Wood, Trunk

config = Config()

def parse() -> list[Wood]:

    # Чтение файлов таблиц
    files_for_parse = list_files_in_directory(config.settings.parse_directory)
    raw_woods = []
    for file in files_for_parse:
        wood_of_file = XLSParser().parse(file)
        raw_woods.extend(wood_of_file)

    woods = []
    # Парсинг таблиц
    for wood_list in raw_woods:
        raw_wood = RawWood(*wood_list)
        try:
            if raw_wood.is_valid():
                for wood in raw_wood.parse():
                    woods.append(wood)
        except Exception as e:
            print(e)

    # TODO: Обработка непрочитанных деревьев

    for wood in woods:
        print(wood)

    return woods

for wood in parse():
    print(wood)

Ошибка валидации в строке: '30-31'	'ива ломкая (поросль) -error'	'30м2'	'2'	'2'
Ошибка в извлечении количества деревьев или площади.
Ошибка валидации в строке: '32-35'	'Ель обыкновенная -error'	'3'	'2,2'	'2,2,2'
Ошибка парсинга строки диаметров стволов.
Ошибка парсинга строки диаметров стволов. Диаметров меньше чем номеров
Ошибка валидации в строке: '36-39'	'Ель обыкновенная -error'	'4'	'2,3,2,4'	'2,2,3'
Ошибка парсинга строки высот стволов.
Ошибка парсинга строки высот стволов. Высот меньше чем стволов
Ошибка валидации в строке: '40-41'	'ива ломкая (поросль) -error'	'30м2'	'2,4'	'2'
Ошибка в извлечении количества деревьев или площади.
Ошибка валидации в строке: '42-44'	'Ель обыкновенная -error'	'4'	'2,3,2,4'	'2,2,3,2'
Ошибка парсинга строки диаметров стволов.
Ошибка парсинга строки диаметров стволов. Диаметров больше чем номеров
1, Береза пушистая, дерево, Диаметр 4.0 см, высота 5.0 м
2, Ель обыкновенная, дерево, Диаметр 4.0 см, высота 5.0 м
3, Ель обыкновенная, дерево, Диаметр 4.0 см

In [40]:
import json
from typing import Literal

class UnknownSpecie(Exception):
    pass

class ParseDensityError(Exception):
    pass

class WriteDensityError(Exception):
    pass

class Density:
    def __init__(self):
        try:
            with open('data/density.json', 'r', encoding='utf-8') as file:
                data = json.load(file)
                self.group_of_wood = data.get('group_of_wood', {})
                self.density_dict = data.get('density_dict', {})
        except Exception as e:
            print("Ошибка чтения файла density.json", e)

    def get_density(self, specie: str) -> int:
        """
        Определяет плотность породы.
        :param specie: порода
        :return: плотность кг/м3
        """
        for group, species_list in self.group_of_wood.items():
            if specie in species_list:
                return self.density_dict.get(group)
        raise UnknownSpecie(f"Попытка получения плотности для неизвестной породы '{specie}'.")

    def add_specie_to_group(self, group: Literal["мягкие", "твердые", "лиственница"], specie: str) -> None:
        """
        Добавляет новый вид древесины в указанный список группы.
        :param group: группа плотности ("мягкие", "твердые" или "лиственница")
        :param specie: порода древесины
        :return: None
        """

        if group not in self.group_of_wood:
            raise ValueError(f"Группа '{group}' не найдена. Доступные группы: "
                             f"{', '.join(self.group_of_wood.keys())}")

        if specie in self.group_of_wood[group]:
            print(f"Порода '{specie}' уже присутствует в группе '{group}'.")
            return

        self.group_of_wood[group].append(specie)
        try:
            with open('data/density.json', 'w', encoding='utf-8') as file:
                json.dump({
                    'group_of_wood': self.group_of_wood,
                    'density_dict': self.density_dict
                }, file, ensure_ascii=False, indent=4)
        except Exception as e:
            print("Ошибка записи в файл density.json", e)



Порода 'можжевельник' уже присутствует в группе 'лиственница'.
